<a href="https://colab.research.google.com/github/silverstar0727/pragmatism-AI/blob/main/W%26B_keras_sweep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install wandb -q

     |████████████████████████████████| 1.8MB 4.3MB/s 
     |████████████████████████████████| 102kB 13.8MB/s 
     |████████████████████████████████| 143kB 50.3MB/s 
     |████████████████████████████████| 143kB 50.1MB/s 
     |████████████████████████████████| 174kB 46.6MB/s 
     |████████████████████████████████| 71kB 9.4MB/s 
ERROR: requests 2.23.0 has requirement urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you'll have urllib3 1.26.6 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.


In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.datasets import cifar10

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import wandb
from wandb.keras import WandbCallback

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.6) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [3]:
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [4]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

11493376/11490434 [==============================] - 0s 0us/step


In [5]:
AUTO = tf.data.experimental.AUTOTUNE
IMG_SHAPE = 32
CHANNELS = 3
NUM_CLASSES = 10
VAL_BATCH_SIZE = 64

In [6]:
class Model(tf.keras.Model):
    def __init__(self):
        super(Model, self).__init__()
        self.flatten = tf.keras.layers.Flatten(input_shape=(28, 28))
        self.linear1 = tf.keras.layers.Dense(128, activation='relu')
        self.dropout = tf.keras.layers.Dropout(0.2)
        self.linear2 = tf.keras.layers.Dense(10, activation='softmax')

    def call(self, x):
        x = self.flatten(x)
        x = self.linear1(x)
        x = self.dropout(x)
        x = self.linear2(x)

        return x

In [7]:
def train():
    # Specify the hyperparameter to be tuned along with
    # an initial value
    config_defaults = {
        'batch_size': 8,
        'learning_rate': 0.01
    }

    # Initialize wandb with a sample project name
    wandb.init(config=config_defaults)

    # Specify the other hyperparameters to the configuration, if any
    wandb.config.epochs = 25

    # Prepare trainloader
    trainloader = tf.data.Dataset.from_tensor_slices((x_train, y_train))
    trainloader = trainloader.shuffle(1024).batch(wandb.config.batch_size).prefetch(tf.data.experimental.AUTOTUNE)
    # prepare testloader 
    testloader = tf.data.Dataset.from_tensor_slices((x_test, y_test))
    testloader = testloader.batch(wandb.config.batch_size).prefetch(tf.data.experimental.AUTOTUNE)

    # Iniialize model with hyperparameters
    keras.backend.clear_session()
    
    model = Model()

    # Compile the model
    opt = tf.keras.optimizers.Adam(learning_rate=wandb.config.learning_rate) # optimizer with different learning rate specified by config
    model.compile(opt, 'sparse_categorical_crossentropy', metrics=['acc'])
    
    # Train the model
    _ = model.fit(trainloader,
                  epochs=wandb.config.epochs, 
                  validation_data=testloader,
                  callbacks=[WandbCallback()]) # WandbCallback to automatically track metrics
                            
    # Evaluate    
    loss, accuracy = model.evaluate(testloader, callbacks=[WandbCallback()])
    print('Test Error Rate: ', round((1-accuracy)*100, 2))
    wandb.log({'Test Error Rate': round((1-accuracy)*100, 2)}) # wandb.log to track custom metrics

In [8]:
sweep_config = {
  'method': 'bayes', 
  'metric': {
      'name': 'val_loss',
      'goal': 'minimize'
  },
  'early_terminate':{
      'type': 'hyperband',
      'min_iter': 5
  },
  'parameters': {
      'batch_size': {
          'values': [8, 16, 32, 64, 128, 256]
      },
      'learning_rate':{
          'values': [0.01, 0.005, 0.001, 0.0005, 0.0001]
      }
  }
}

In [9]:
sweep_id = wandb.sweep(sweep_config, project="abc")

Create sweep with ID: 4936c6nf
Sweep URL: https://wandb.ai/jeongmin/abc/sweeps/4936c6nf


In [10]:
wandb.agent(sweep_id, function=train)

wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: ihps7vb5 with config:
wandb: 	batch_size: 32
wandb: 	learning_rate: 0.001
wandb: Currently logged in as: jeongmin (use `wandb login --relogin` to force relogin)


Epoch 1/25
1875/1875 [==============================] - 6s 2ms/step - loss: 0.2879 - acc: 0.9173 - val_loss: 0.1325 - val_acc: 0.9594


wandb: ERROR Can't save model, h5py returned error: Saving the model to HDF5 format requires the model to be a Functional model or a Sequential model. It does not work for subclassed models, because such models are defined via the body of a Python method, which isn't safely serializable. Consider saving to the Tensorflow SavedModel format (by setting save_format="tf") or using `save_weights`.


Epoch 2/25
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1385 - acc: 0.9588 - val_loss: 0.0992 - val_acc: 0.9701
Epoch 3/25
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1033 - acc: 0.9685 - val_loss: 0.0852 - val_acc: 0.9741
Epoch 4/25
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0853 - acc: 0.9742 - val_loss: 0.0775 - val_acc: 0.9757
Epoch 5/25
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0723 - acc: 0.9766 - val_loss: 0.0721 - val_acc: 0.9769
Epoch 6/25
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0649 - acc: 0.9793 - val_loss: 0.0769 - val_acc: 0.9765
Epoch 7/25
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0555 - acc: 0.9819 - val_loss: 0.0677 - val_acc: 0.9787
Epoch 8/25
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0516 - acc: 0.9826 - val_loss: 0.0730 - val_acc: 0.9773
Epoch 9/25
1875/1875 [==============================] - 4s 2ms

epoch,24
loss,0.02255
acc,0.9923
val_loss,0.08446
val_acc,0.9808
_runtime,127
_timestamp,1626691987
_step,25
best_val_loss,0.06539
best_epoch,9
Test Error Rate,1.92


...

wandb: Agent Starting Run: 7jn4q6oc with config:
wandb: 	batch_size: 256
wandb: 	learning_rate: 0.0001


Epoch 1/25
235/235 [==============================] - 1s 4ms/step - loss: 1.3773 - acc: 0.6350 - val_loss: 0.7329 - val_acc: 0.8435


wandb: ERROR Can't save model, h5py returned error: Saving the model to HDF5 format requires the model to be a Functional model or a Sequential model. It does not work for subclassed models, because such models are defined via the body of a Python method, which isn't safely serializable. Consider saving to the Tensorflow SavedModel format (by setting save_format="tf") or using `save_weights`.


Epoch 2/25
103/235 [============>.................] - ETA: 0s - loss: 0.7118 - acc: 0.8207

wandb: Ctrl + C detected. Stopping sweep.
